In [1]:
import mlxtend
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from mlxtend.preprocessing import TransactionEncoder

In [3]:
data = np.array([
    ['우유', '기저귀', '쥬스'],
    ['양상추', '기저귀', '맥주'],
    ['우유', '양상추', '기저귀', '쥬스'],
    ['양상추', '맥주']
])

In [4]:
te = TransactionEncoder()

In [5]:
te_ary = te.fit(data).transform(data)
print(te_ary)

[[ True False False  True  True]
 [ True  True  True False False]
 [ True False  True  True  True]
 [False  True  True False False]]


In [6]:
df = pd.DataFrame(te_ary, columns=te.columns_)

In [7]:
df

,기저귀,맥주,양상추,우유,쥬스
0,True,False,False,True,True
1,True,True,True,False,False
2,True,False,True,True,True
3,False,True,True,False,False


In [8]:
from mlxtend.frequent_patterns import apriori

In [12]:
result = apriori(df, min_support=0.5, use_colnames=True)
result

,support,itemsets
0,0.75,(기저귀)
1,0.50,(맥주)
2,0.75,(양상추)
3,0.50,(우유)
4,0.50,(쥬스)
5,0.50,"(양상추, 기저귀)"
6,0.50,"(우유, 기저귀)"
7,0.50,"(쥬스, 기저귀)"
8,0.50,"(맥주, 양상추)"
9,0.50,"(쥬스, 우유)"


In [ ]:
## 쥬스+우유를 함께 산 사람에게 추천을 해준다고 한다면, 
## 맥주보다는 기저귀를 추천해야함.
## 지지도 기반으로 판단!

In [11]:
apriori(df, min_support=0.7, use_colnames=True)

,support,itemsets
0,0.75,(기저귀)
1,0.75,(양상추)


In [13]:
type(result) #result = apriori(df, min_support=0.5, use_colnames=True)

pandas.core.frame.DataFrame

In [ ]:
# result에서 마지막 itemsets컬럼값 추출
## 쥬스, 우유와 함께 추천하면 좋을 item은??

In [18]:
choice = result.iloc[10, :]
choice

support               0.5
itemsets    (쥬스, 우유, 기저귀)
Name: 10, dtype: object

In [20]:
items = choice['itemsets']
items

frozenset({'기저귀', '우유', '쥬스'})

In [23]:
list(items)[2]

'기저귀'

In [ ]:
## one-hot인코딩 방식의 테이블이 필요!!! 
## 실제로는 물건의 개수가 아주 많음.
## item 수 => 1조: 38441개의 여행지, 2조: 388개의 물건
## user 수 => 1조: 70000명, 2조: 800명
## 1조: 70000 * 38441, 2조: 800 * 388
## 1조인 경우 한 사람이 간 곳이 10개라고 한다면, 나머지는 0
## ==> 0이 2차원 데이터: 희소행렬(sparse matrix)
## TF-IDF, CounterVectorizor, TransactionEncoder

In [25]:
#!pip install gensim

In [26]:
from gensim.models import Word2Vec

In [27]:
## 자연어에서 특정한 단어(word)를 중심으로 앞 뒤의 단어를 유추(예측, 추론)할 때 사용됨.

In [28]:
doc = [
    'you say goodbye and I say hello', 
    'i say happy or he said white',
    'we say unhappy and angry black'
]

In [29]:
doc2 = [
    '나는 굿바이라고 했고, 너는 헬로우라고 했어.', 
    '나는 기쁘다고 했고, 너는 하얗게 슬프다고 했어.',
    '우리는 행복하지 않고, 블랙으로 슬퍼'
]

In [30]:
##문장을 토큰으로 끊어내자.
sentences = [sentence.split(' ') for sentence in doc]
sentences

[['you', 'say', 'goodbye', 'and', 'I', 'say', 'hello'],
 ['i', 'say', 'happy', 'or', 'he', 'said', 'white'],
 ['we', 'say', 'unhappy', 'and', 'angry', 'black']]

In [31]:
sentences2 = [sentence.split(' ') for sentence in doc2]
sentences2

[['나는', '굿바이라고', '했고,', '너는', '헬로우라고', '했어.'],
 ['나는', '기쁘다고', '했고,', '너는', '하얗게', '슬프다고', '했어.'],
 ['우리는', '행복하지', '않고,', '블랙으로', '슬퍼']]

In [33]:
model = Word2Vec(window=1, min_count=1) ##특정한 단어의 앞뒤 단어를 유추가능.
## window: 유추하는 계산을 할 때 특정한 단어의 앞뒤 몇개를 가지고 계산을 할지의 단위
## min_count: 특정한 패턴의 출현 횟수

In [38]:
model2 = Word2Vec(window=1, min_count=1)
model2.build_vocab(sentences2)

In [34]:
model.build_vocab(sentences) #영문문장을 가지고 계산을 함.

In [35]:
## 특정한 단어를 주고 앞뒤 단어를 추론해보자.
model.wv.most_similar('say') ##비지도!!

[('or', 0.21617144346237183),
 ('hello', 0.0931011363863945),
 ('said', 0.09291724115610123),
 ('i', 0.07963486015796661),
 ('happy', 0.06285081058740616),
 ('you', 0.05433368682861328),
 ('he', 0.0270574688911438),
 ('white', 0.0161347147077322),
 ('and', -0.010839186608791351),
 ('unhappy', -0.027750365436077118)]

In [36]:
model.wv.most_similar('or')

[('say', 0.21617144346237183),
 ('he', 0.044689204543828964),
 ('happy', 0.015025186352431774),
 ('you', 0.010695194825530052),
 ('white', 0.001951059210114181),
 ('black', -0.03284314647316933),
 ('goodbye', -0.0455227792263031),
 ('hello', -0.045689091086387634),
 ('i', -0.0742427185177803),
 ('said', -0.09326909482479095)]

In [ ]:
## or단어 앞으로는 say가 나올 가능성이 제일 크고,
## or단어 뒤로는 black단어가 나올 가능성이 제일 큰 것을 알 수 있다.

In [40]:
model2.wv.most_similar('너는')

[('굿바이라고', 0.13725271821022034),
 ('나는', 0.06797593832015991),
 ('우리는', 0.03364057093858719),
 ('블랙으로', 0.009391184896230698),
 ('기쁘다고', 0.00831596553325653),
 ('슬퍼', 0.004503015894442797),
 ('헬로우라고', -0.0036444757133722305),
 ('했어.', -0.010839187540113926),
 ('했고,', -0.023671669885516167),
 ('슬프다고', -0.09575346857309341)]

In [41]:
## '너는' 한글 단어 앞으로 나올 단어로는 굿바이라고를 추천,
## '너는' 한글 단어 뒤로 나올 단어로는 헬로우라고를 추천

In [42]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [44]:
tfidf = TfidfVectorizer(stop_words='english') #불용어처리가능한 ifidf객체 생성
##tfidf는 빈도수를 구하긴 하지만, 문서마다 자주 등장하는 문서는 각 문서를 구분할 단어로 사용하기는
##부적합하여 가중치를 낮추어 계산

In [45]:
tfidf_matrix = tfidf.fit_transform(doc)
tfidf_matrix

<3x9 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [46]:
## 3 x 9에서
## 3은 문서의 개수
## 9는 단어의 개수

In [47]:
tfidf_matrix.toarray()

array([[0.        , 0.        , 0.54270061, 0.        , 0.54270061,
        0.        , 0.64105545, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.54645401, 0.        ,
        0.54645401, 0.32274454, 0.        , 0.54645401],
       [0.54645401, 0.54645401, 0.        , 0.        , 0.        ,
        0.        , 0.32274454, 0.54645401, 0.        ]])